<a href="https://colab.research.google.com/github/mtzig/LIDC_GDRO/blob/main/transfer_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [169]:
!git clone https://github.com/mtzig/LIDC_GDRO.git
%cd /content/LIDC_GDRO

fatal: destination path 'LIDC_GDRO' already exists and is not an empty directory.
/content/LIDC_GDRO


In [170]:
!git pull

Already up to date.


In [171]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [172]:
import os
import torch
import pandas as pd
import numpy as np
from datasets import NoduleDataset
from dataloaders import SubtypedDataLoader

In [173]:
if torch.cuda.is_available():
    print("Good to go!")
    DEVICE = torch.device("cuda")
else:
    print("Using cpu")
    DEVICE = torch.device("cpu")

Good to go!


In [174]:
lidc_subtyped = pd.read_csv('./data/lidc_spic_subgrouped.csv', index_col=0)
lidc_subtyped.head(5)

,noduleID,malignancy,spiculation,malignancy_b,spiculation_b,subgroup
0,1,5.0,5.0,malignant,marked,marked_malignant
1,2,5.0,1.0,malignant,unmarked,unmarked_malignant
2,3,5.0,2.0,malignant,marked,marked_malignant
3,4,4.0,2.0,malignant,marked,marked_malignant
4,5,5.0,1.0,malignant,unmarked,unmarked_malignant


In [175]:
lidc_subtyped.groupby(['spiculation_b', 'malignancy_b']).size()

spiculation_b  malignancy_b
marked         benign          101
               malignant       407
unmarked       benign          725
               malignant       282
dtype: int64

In [176]:
def getNormed(this_array, this_min = 0, this_max = 255, set_to_int = True):
    
    rat = (this_max - this_min)/(this_array.max() - this_array.min())
    this_array = this_array * rat
    this_array -= this_array.min()
    this_array += this_min
    if set_to_int:
        return this_array.to(dtype= torch.int)
    return this_array

In [177]:
def getImages(image_folder):
    '''
        Input:
        image_folder: directory of the image files

        Output:
        m1: list of the labels encountered (1,2,4,5)
        m2: list of binary labels encountered (benign, malignant)
        diff: list of any nodes with discrepency to CSV labels

    '''
    
    train_img = []
    train_label = []
    train_spic_label = []

    t_marked_benign = []
    t_unmarked_benign = []
    
    t_marked_malignant = []
    t_unmarked_malignant = []




    marked_benign = []
    unmarked_benign = []
    
    marked_malignant = []
    unmarked_malignant = []

    

    lidc = pd.read_csv('./data/lidc_spic_subgrouped.csv')
    train_test = pd.read_csv('./data/lidc_train_test_split_stratified.csv')
    for dir1 in os.listdir(image_folder):
  
        if dir1 == 'Malignancy_3':
            continue

        for file in os.listdir(os.path.join(image_folder, dir1)):


            temp_nodule_ID = file.split('.')[0]
            subtype = lidc[lidc['noduleID']==int(temp_nodule_ID)]['subgroup'].iloc[0]
            malignancy = lidc[lidc['noduleID']==int(temp_nodule_ID)]['malignancy'].iloc[0]
            spiculation = lidc[lidc['noduleID']==int(temp_nodule_ID)]['malignancy'].iloc[0]
            
            train_type = train_test[train_test['noduleID'] ==int(temp_nodule_ID)]['dataset'].iloc[0]
            
            
            image = np.loadtxt(os.path.join(image_folder, dir1,file))
            image = torch.from_numpy(image).to(DEVICE)
            rgb_image = torch.stack((image,image,image), dim = 0)
            rgb_image = getNormed(rgb_image)
            rgb_image = rgb_image / 255 

            
            
            if train_type == 'train':
                train_img.append(rgb_image)
                train_label.append(torch.tensor(1).to(DEVICE).to(torch.float32) if malignancy > 3 else torch.tensor(0).to(DEVICE).to(torch.float32))
                train_spic_label.append(torch.tensor(1).to(DEVICE).to(torch.float32) if spiculation > 1 else torch.tensor(0).to(DEVICE).to(torch.float32))


                if subtype == 'marked_benign':
                  image_array = t_marked_benign
                elif subtype == 'unmarked_benign':
                  image_array = t_unmarked_benign
                elif subtype == 'marked_malignant':
                  image_array = t_marked_malignant
                else:
                  image_array = t_unmarked_malignant
            
                image_array.append(rgb_image)
                
                continue
            
            if subtype == 'marked_benign':
                image_array = marked_benign
            elif subtype == 'unmarked_benign':
                image_array = unmarked_benign
            elif subtype == 'marked_malignant':
                image_array = marked_malignant
            else:
                image_array = unmarked_malignant
            
            image_array.append(rgb_image)
 
    train_img_gdro = [(t_marked_benign, torch.tensor(0).to(DEVICE).to(torch.float32)), 
                      (t_unmarked_benign,torch.tensor(0).to(DEVICE).to(torch.float32)), 
                      (t_marked_malignant, torch.tensor(1).to(DEVICE).to(torch.float32)), 
                      (t_unmarked_malignant, torch.tensor(1).to(DEVICE).to(torch.float32))]

    return train_img, train_label, train_img_gdro, train_spic_label, marked_benign, unmarked_benign, marked_malignant, unmarked_malignant


In [178]:
train_img, train_label, train_img_gdro, train_spic_label, marked_benign, unmarked_benign, marked_malignant, unmarked_malignant = getImages('./LIDC(MaxSlices)_Nodules(fixed)')

In [179]:
sum(train_spic_label)

tensor(949., device='cuda:0')

In [180]:
len(train_img)

1210

In [181]:
len(marked_benign)

21

In [182]:
len(unmarked_benign)

145

In [183]:
len(marked_malignant)

82

In [184]:
len(unmarked_malignant)

57

#model train!!!

In [185]:
from models import VGGNet
from loss import ERMLoss, GDROLoss
from datasets import NoduleDataset
from dataloaders import InfiniteDataLoader

import train

In [186]:
model = VGGNet(device=DEVICE)

In [187]:
model

VGGNet(
  (model): VGG(
    (features): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU(inplace=True)
      (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (6): ReLU(inplace=True)
      (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): ReLU(inplace=True)
      (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace=True)
      (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (13): ReLU(inplace=True)
      (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (15): ReLU(inplace=True)
      (16): C

In [188]:
loss_fn = ERMLoss(model,torch.nn.functional.binary_cross_entropy_with_logits,{}) #torch.nn.CrossEntropyLoss()

In [189]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.0005, weight_decay=0.005)


In [190]:
train_dataset = NoduleDataset(train_img, train_label) #train_spic_label

In [191]:
len(train_dataset)

1210

In [192]:
# train_set, val_set = torch.utils.data.random_split(train_dataset, [1000, 210])

In [193]:
train_loader = InfiniteDataLoader(train_dataset, 1210)#InfiniteDataLoader(train_set, 1000)

In [194]:
# val_loader = InfiniteDataLoader(val_set, 210)

In [195]:
epochs = 40
for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")
    train.train(train_loader, model, loss_fn, optimizer, verbose=True)
                # train.test(test_dataloader, model)

    # model.eval()
    # with torch.no_grad():
    #   X,y = next(val_loader)
    #   results = torch.sigmoid(model(X))
    #   accuracy = torch.sum(torch.round(results) == y)/210
    # model.train()
    # print(f'cv accuracy {accuracy}')

Epoch 1/40
Average training loss: 0.7038122415542603
Epoch 2/40
Average training loss: 0.811580240726471
Epoch 3/40
Average training loss: 0.6902214288711548
Epoch 4/40
Average training loss: 0.6799570918083191
Epoch 5/40
Average training loss: 0.6678031086921692
Epoch 6/40
Average training loss: 0.6507728695869446
Epoch 7/40
Average training loss: 0.6293487548828125
Epoch 8/40
Average training loss: 0.6177820563316345
Epoch 9/40
Average training loss: 0.5920219421386719
Epoch 10/40
Average training loss: 0.580757737159729
Epoch 11/40
Average training loss: 0.555697500705719
Epoch 12/40
Average training loss: 0.5383449196815491
Epoch 13/40
Average training loss: 0.5293784737586975
Epoch 14/40
Average training loss: 0.49145573377609253
Epoch 15/40
Average training loss: 0.45866215229034424
Epoch 16/40
Average training loss: 0.4743494987487793
Epoch 17/40
Average training loss: 0.4337193965911865
Epoch 18/40
Average training loss: 0.41891857981681824
Epoch 19/40
Average training loss: 0.

#ERM accuracies

##marked_benign

In [196]:
len(marked_benign)

21

In [197]:
results = torch.sigmoid(model(torch.stack(marked_benign).to(DEVICE)))
accuracy = torch.sum(torch.round(results) == torch.zeros(len(marked_benign), device=DEVICE))/len(marked_benign)
accuracy

tensor(0.7619, device='cuda:0')

##unmarked_benign

In [198]:
len(unmarked_benign)

145

In [199]:
results = torch.sigmoid(model(torch.stack(unmarked_benign).to(DEVICE)))
accuracy = torch.sum(torch.round(results) == torch.zeros(len(unmarked_benign), device=DEVICE))/len(unmarked_benign)
accuracy

tensor(0.8828, device='cuda:0')

##marked_malignant



In [200]:
results = torch.sigmoid(model(torch.stack(marked_malignant).to(DEVICE)))
accuracy = torch.sum(torch.round(results) == torch.ones(len(marked_malignant), device=DEVICE))/len(marked_malignant)
accuracy

tensor(0.6463, device='cuda:0')

In [201]:
len(marked_malignant)

82

##unmarked malignant

In [202]:
len(unmarked_malignant)

57

In [203]:
results = torch.sigmoid(model(torch.stack(unmarked_malignant).to(DEVICE)))
accuracy = torch.sum(torch.round(results) == torch.ones(len(unmarked_malignant), device=DEVICE))/len(unmarked_malignant)
accuracy

tensor(0.4737, device='cuda:0')

#GDRO

In [204]:
train_loader = SubtypedDataLoader(train_img_gdro, 1210, True, True)

In [205]:
# model = VGGNet(device=DEVICE)

In [206]:
loss_fn = GDROLoss(model,torch.nn.functional.binary_cross_entropy_with_logits,{"groupdro_eta":0.001},True) #torch.nn.CrossEntropyLoss()

In [207]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.0005, weight_decay=0.005)


In [208]:
epochs = 40
for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")
    train.train(train_loader, model, loss_fn, optimizer, verbose=True)
                # train.test(test_dataloader, model)

    # model.eval()
    # with torch.no_grad():
    #   X,y = next(val_loader)
    #   results = torch.sigmoid(model(X))
    #   accuracy = torch.sum(torch.round(results) == y)/210
    # model.train()
    # print(f'cv accuracy {accuracy}')

Epoch 1/40
Average training loss: 0.024052973836660385
Epoch 2/40
Average training loss: 1.133148193359375
Epoch 3/40
Average training loss: 0.07945780456066132
Epoch 4/40
Average training loss: 0.2556660771369934
Epoch 5/40
Average training loss: 0.2407134473323822
Epoch 6/40
Average training loss: 0.16123875975608826
Epoch 7/40
Average training loss: 0.1224861741065979
Epoch 8/40
Average training loss: 0.1212521642446518
Epoch 9/40
Average training loss: 0.12827539443969727
Epoch 10/40
Average training loss: 0.13510823249816895
Epoch 11/40
Average training loss: 0.1302674114704132
Epoch 12/40
Average training loss: 0.12443814426660538
Epoch 13/40
Average training loss: 0.10933086276054382
Epoch 14/40
Average training loss: 0.10115789622068405
Epoch 15/40
Average training loss: 0.09535400569438934
Epoch 16/40
Average training loss: 0.09908991307020187
Epoch 17/40
Average training loss: 0.09788673371076584
Epoch 18/40
Average training loss: 0.0995715856552124
Epoch 19/40
Average traini

#GDRO accuracies

##marked_benign

In [209]:
len(marked_benign)

21

In [210]:
results = torch.sigmoid(model(torch.stack(marked_benign).to(DEVICE)))
accuracy = torch.sum(torch.round(results) == torch.zeros(len(marked_benign), device=DEVICE))/len(marked_benign)
accuracy

tensor(0.7619, device='cuda:0')

##unmarked_benign

In [211]:
len(unmarked_benign)

145

In [212]:
results = torch.sigmoid(model(torch.stack(unmarked_benign).to(DEVICE)))
accuracy = torch.sum(torch.round(results) == torch.zeros(len(unmarked_benign), device=DEVICE))/len(unmarked_benign)
accuracy

tensor(0.8690, device='cuda:0')

##marked_malignant



In [213]:
results = torch.sigmoid(model(torch.stack(marked_malignant).to(DEVICE)))
accuracy = torch.sum(torch.round(results) == torch.ones(len(marked_malignant), device=DEVICE))/len(marked_malignant)
accuracy

tensor(0.6585, device='cuda:0')

In [214]:
len(marked_malignant)

82

##unmarked malignant

In [215]:
len(unmarked_malignant)

57

In [216]:
results = torch.sigmoid(model(torch.stack(unmarked_malignant).to(DEVICE)))
accuracy = torch.sum(torch.round(results) == torch.ones(len(unmarked_malignant), device=DEVICE))/len(unmarked_malignant)
accuracy

tensor(0.5088, device='cuda:0')